## Домашнее задание 2 

### связь Python с программами на C/C++

1. Реализовать на языке C/C++ классические операции перемножения квадратных матриц и умножения матрицы на вектор (15%)

In [2]:
# можно использовать %%writefile и оформить сишный код прямо здесь
# но наверное удобней будет писать его напрямую в отдельных файлах 
# и прислать вместе с домашним заданием

2. Разделить программу на несколько модулей и провести сборку через статическую линковку (25%)

In [ ]:
# можно используя знак ! запустить команду сборки прямо отсюда (так будет видно, что пункт 2 выполнен

3. Подготовьте две сборки с флагами -g и -O3 и измерьте времена выполнения операций с N = 512, 1024, . . ., 4096 (20%)

In [ ]:
# замеры можно сделать используя %%time, %%timeit
# если комфортней работать с терминалом, то можно приложить 
# к домашнему заданию скрин с записанными временами расчетов

4. Выполните вызов процедуры из Python через Ctypes/Cython/PyBind11 и измерьте времена (40%)

In [3]:
# можно это сделать здесь

БОНУСЫ:

1. Дополнительные баллы за использование функций BLAS/cBLAS/openBLAS;
2. Дополнительные баллы за вызов теста LINPACK на вашем компьютере;
3. Супербонус: реализовать метод Штрассена перемножения квадратных матриц